In [1]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-65-2-83-175.ap-south-1.compute.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp - 11 ML Algos with HP Tuning")

2024/10/14 17:07:28 INFO mlflow.tracking.fluent: Experiment with name 'Exp - 11 ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://youtube-comments-sentiment/938969902056822744', creation_time=1728905867310, experiment_id='938969902056822744', last_update_time=1728905867310, lifecycle_stage='active', name='Exp - 11 ML Algos with HP Tuning', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [4]:
df = pd.read_csv('Preprocessed_Reddit.csv').dropna()
df.shape

(36662, 2)

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Random Forest

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)  # Number of trees in the forest
    max_depth = trial.suggest_int('max_depth', 3, 20)  # Maximum depth of the tree
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  # Minimum samples required to split a node
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)  # Minimum samples required at a leaf node

    # RandomForestClassifier setup
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Random Forest, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rf, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'],
                                        random_state=42)

    # Log the best model with MLflow, passing the algo_name as "RandomForest"
    log_mlflow("RandomForest", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Random Forest
run_optuna_experiment()

[I 2024-10-14 17:08:34,970] A new study created in memory with name: no-name-f0128151-ad8c-41fd-bbd8-43484e27715a
[I 2024-10-14 17:08:40,132] Trial 0 finished with value: 0.6382371591629676 and parameters: {'n_estimators': 194, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.6382371591629676.
